In [1]:
import os
import copy
import time
import asyncio
import warnings
import logging

from pathlib import Path

import numpy as np
from astropy.io import fits

from scipy import ndimage
from scipy.signal import medfilt
from scipy.ndimage.filters import gaussian_filter
from astropy.modeling import models, fitting


import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [7, 6]

#Import CWFS package
from lsst import cwfs
from lsst.cwfs.instrument import Instrument
from lsst.cwfs.algorithm import Algorithm
from lsst.cwfs.image import Image, readFile, aperture2image, showProjection
import lsst.cwfs.plots as plots

%matplotlib inline

In [2]:
import wget

In [3]:
from lsst.ts import salobj
from lsst.ts.standardscripts.auxtel.attcs import ATTCS
import asyncio

In [4]:
attcs = ATTCS()
cam = salobj.Remote(attcs.domain, "GenericCamera", 1)

In [5]:
await asyncio.gather(attcs.start_task, cam.start_task) 

RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 100 messages
RemoteTelemetry(ATDome, 0, position) falling behind; read 23 messages


[[None, None, None, None, None, None, None], None]

RemoteEvent(ATAOS, 0, logMessage) falling behind; read 11 messages


## Declare target

In [7]:
# Declare target name
target_name='etaLeo'

In [47]:
dz=1.5
focus_offset = (328.90741298-121.19959549+310.44675607-424.60521034-439.96161154-1.06777733e+02+78.0112456)/1e3/4.2
expTime=30.0
print(focus_offset)

-0.0893282704095238


In [9]:
focus_offset

0.0

In [48]:
attcs.athexapod.evt_positionUpdate.flush()
await attcs.ataos.cmd_applyAxisOffset.set_start(axis="z", offset=focus_offset)
try:
    curr_hex_pos = await attcs.athexapod.evt_positionUpdate.next(flush=False, timeout=30)
except asyncio.TimeoutError:
    print("Timedout")

In [ ]:
attcs.athexapod.evt_positionUpdate.flush()
await attcs.ataos.cmd_applyAxisOffset.set_start(axis="x", offset=-0.6881088203816793)
try:
    curr_hex_pos = await attcs.athexapod.evt_positionUpdate.next(flush=False, timeout=30)
except asyncio.TimeoutError:
    print("Timedout")

In [ ]:
await cam.cmd_startLiveView.set_start(expTime=0.5)

In [51]:
await attcs.ataos.cmd_applyAxisOffset.set_start(axis="m1", offset=0.e3)

RemoteEvent(ATMCS, 0, target) falling behind; read 100 messages
RemoteEvent(ATDomeTrajectory, 0, logMessage) falling behind; read 39 messages


In [49]:
try:
    await cam.cmd_stopLiveView.start()
except salobj.AckError:
    print("ack error")

# Intrafocal hexapod setting - offsets -0.5mm from central focus value above

print('Move to intrafocal position')
# await attcs.athexapod.cmd_moveToPosition.set_start(x=hex_X, y=hex_Y,
#                                              z=hex_Z-dz, u=hex_U, v=hex_V)
attcs.athexapod.evt_positionUpdate.flush()
await attcs.ataos.cmd_applyAxisOffset.set_start(axis="z", offset=-dz+focus_offset)
curr_hex_pos = await attcs.athexapod.evt_positionUpdate.next(flush=False, timeout=30)

print('Take intrafocal image')
cam.evt_endReadout.flush()

await cam.cmd_takeImages.set_start(numImages=1, expTime=expTime, shutter=True, imageSequenceName=target_name+'_intra')

intra_endron = await cam.evt_endReadout.next(flush=False, timeout=1)

# Extrafocal hexapod setting
print('Move to extra-focal hexapod position')
# await attcs.athexapod.cmd_moveToPosition.set_start(x=hex_X, y=hex_Y,
#                                                      z=hex_Z+dz, u=hex_U, v=hex_V)
attcs.athexapod.evt_positionUpdate.flush()
await attcs.ataos.cmd_applyAxisOffset.set_start(axis="z", offset=dz+focus_offset)
curr_hex_pos = await attcs.athexapod.evt_positionUpdate.next(flush=False, timeout=30)

# r.evt_endReadout.flush()
print('Take extra-focal image')
cam.evt_endReadout.flush()
await cam.cmd_takeImages.set_start(numImages=1, expTime=expTime, shutter=True, imageSequenceName=target_name+'_extra')

extra_endron = await cam.evt_endReadout.next(flush=False, timeout=1)

print(f"============")
print(intra_endron.imageName)
print(extra_endron.imageName)
print(f"============")

try:
    await cam.cmd_startLiveView.set_start(expTime=0.5)
except salobj.AckError:
    print("ack error")

attcs.athexapod.evt_positionUpdate.flush()
await attcs.ataos.cmd_applyAxisOffset.set_start(axis="z", offset=focus_offset)
curr_hex_pos = await attcs.athexapod.evt_positionUpdate.next(flush=False, timeout=30)

# intra_wget_url = 'http://139.229.170.216:8000/data/'+intra_endron.imageName+'.fits'
# extra_wget_url = 'http://139.229.170.216:8000/data/'+extra_endron.imageName+'.fits'
# intra_fname = wget.download(intra_wget_url)
# print(f"============")
# print(f'{intra_fname}')
# extra_fname = wget.download(extra_wget_url)
# print(f'{extra_fname}')
# print(f"============")

Move to intrafocal position
Take intrafocal image
Move to extra-focal hexapod position
Take extra-focal image
1579851935-etaLeo_intra-0-1
1579851975-etaLeo_extra-0-1
